In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_rerun_links.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 2)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 1737
driver_count = 0
driver = get_driver()
for url in url_list[url_count:2000]:
    print(url)
    if driver_count >=20:
        driver_count = 0
        print("need to close driver")
        driver.quit()
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_1k_2k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    driver.delete_all_cookies()
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6793649520543723520-4
Page load happened
1737 https://kw.com/agent/UPA-6793649520543723520-4
***********************************
https://kw.com/agent/UPA-6587384964894851074-3
Page load happened
1738 https://kw.com/agent/UPA-6587384964894851074-3
***********************************
https://kw.com/agent/UPA-6587385370394263552-3
Page load happened
1739 https://kw.com/agent/UPA-6587385370394263552-3
***********************************
https://kw.com/agent/UPA-6587384964084588544-0
Page load happened
1740 https://kw.com/agent/UPA-6587384964084588544-0
***********************************
https://kw.com/agent/UPA-6592960627190657026-9
Page load happened
1741 https://kw.com/agent/UPA-6592960627190657026-9
***********************************
https://kw.com/agent/UPA-6587384978583121923-7
Page load happened
1742 https://kw.com/agent/UPA-6587384978583121923-7
***********************************
https://kw.com/agent/UPA-6587385194464182273-5
Page load happened
1743 https

Timeout happened no page load
1789 https://kw.com/agent/UPA-6587385432011116545-9
***********************************
https://kw.com/agent/UPA-6782476489509175296-1
Page load happened
1790 https://kw.com/agent/UPA-6782476489509175296-1
***********************************
https://kw.com/agent/UPA-6587385375985983492-3
Page load happened
1791 https://kw.com/agent/UPA-6587385375985983492-3
***********************************
https://kw.com/agent/UPA-6587385233824645120-4
Page load happened
1792 https://kw.com/agent/UPA-6587385233824645120-4
***********************************
https://kw.com/agent/UPA-6704878681985507328-0
Page load happened
1793 https://kw.com/agent/UPA-6704878681985507328-0
***********************************
https://kw.com/agent/UPA-6587385187704815620-3
Page load happened
1794 https://kw.com/agent/UPA-6587385187704815620-3
***********************************
https://kw.com/agent/UPA-6790795263701864448-7
Page load happened
1795 https://kw.com/agent/UPA-6790795263701864

Page load happened
1841 https://kw.com/agent/UPA-6821929695551389696-2
***********************************
https://kw.com/agent/UPA-6605415560119156737-5
Page load happened
1842 https://kw.com/agent/UPA-6605415560119156737-5
***********************************
https://kw.com/agent/UPA-6587385205613375489-8
Page load happened
1843 https://kw.com/agent/UPA-6587385205613375489-8
***********************************
https://kw.com/agent/UPA-6587385315132641280-3
Page load happened
1844 https://kw.com/agent/UPA-6587385315132641280-3
***********************************
https://kw.com/agent/UPA-6833556960288157696-1
Page load happened
1845 https://kw.com/agent/UPA-6833556960288157696-1
***********************************
https://kw.com/agent/UPA-6587385424918720518-2
Page load happened
1846 https://kw.com/agent/UPA-6587385424918720518-2
***********************************
https://kw.com/agent/UPA-6765368090029346816-7
Page load happened
1847 https://kw.com/agent/UPA-6765368090029346816-7
*****

1893 https://kw.com/agent/UPA-6729445787457581056-1
***********************************
https://kw.com/agent/UPA-6587385405686841345-1
Page load happened
1894 https://kw.com/agent/UPA-6587385405686841345-1
***********************************
https://kw.com/agent/UPA-6587385180053815296-2
Page load happened
1895 https://kw.com/agent/UPA-6587385180053815296-2
***********************************
https://kw.com/agent/UPA-6587385326545932292-5
Page load happened
1896 https://kw.com/agent/UPA-6587385326545932292-5
***********************************
https://kw.com/agent/UPA-6834552923992858624-3
need to close driver
new driver initialize****************
Page load happened
1897 https://kw.com/agent/UPA-6834552923992858624-3
***********************************
https://kw.com/agent/UPA-6587385280722464769-2
Page load happened
1898 https://kw.com/agent/UPA-6587385280722464769-2
***********************************
https://kw.com/agent/UPA-6764657825963794432-9
Page load happened
1899 https://kw.c

Timeout happened no page load
1945 https://kw.com/agent/UPA-6809200646301622272-7
***********************************
https://kw.com/agent/UPA-6592948213242204162-4
Page load happened
1946 https://kw.com/agent/UPA-6592948213242204162-4
***********************************
https://kw.com/agent/UPA-6587385184993316871-5
Page load happened
1947 https://kw.com/agent/UPA-6587385184993316871-5
***********************************
https://kw.com/agent/UPA-6587385181435482112-9
Page load happened
1948 https://kw.com/agent/UPA-6587385181435482112-9
***********************************
https://kw.com/agent/UPA-6587385026403348482-3
Page load happened
1949 https://kw.com/agent/UPA-6587385026403348482-3
***********************************
https://kw.com/agent/UPA-6587385162305630210-7
Page load happened
1950 https://kw.com/agent/UPA-6587385162305630210-7
***********************************
https://kw.com/agent/UPA-6813549603085361152-4
Page load happened
1951 https://kw.com/agent/UPA-6813549603085361

Page load happened
1997 https://kw.com/agent/UPA-6587385001973059587-1
***********************************
https://kw.com/agent/UPA-6587385337032302593-7
Page load happened
1998 https://kw.com/agent/UPA-6587385337032302593-7
***********************************
https://kw.com/agent/UPA-6587385255586107392-5
Page load happened
1999 https://kw.com/agent/UPA-6587385255586107392-5
***********************************
